In [1]:
import io
import os
import shutil
import json
import random
import numpy as np
import pickle
import matplotlib.pyplot as plt
import pandas as pd
import keras
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
# storage_dir = 'D:/UMSI Classes/Fall2018/670/Project/danbooru2017/Selected/'
# test_dir = 'D:/UMSI Classes/Fall2018/670/Project/danbooru2017/Test/'

current = os.getcwd()

storage_dir = os.path.join(current + '/Final model/Training/')
test_dir =  os.path.join(current + '/Final model/Test/')

In [3]:
taglist = ['1girl', 'long_hair', 'blush', 'smile', 'short_hair']

In [4]:
# with open('training.pkl', 'rb') as file:
#     training_data = pickle.load(file)
# with open('test.pkl', 'rb') as file:
#     test_data = pickle.load(file)

with open('trainingv2.pkl', 'rb') as file:
    training_data = pickle.load(file)
with open('testv2.pkl', 'rb') as file:
    test_data = pickle.load(file)

In [5]:
check = [[str(x[0])+'.jpg']+list(x[1]) for x in training_data]
df_check = pd.DataFrame(check, columns = ['filename', '1girl', 'long_hair', 'blush', 'smile', 'short_hair'])

### Preprocessing

In [6]:
formatted_train = [[str(x[0])+'.jpg']+list(x[1]) for x in training_data]
formatted_test = [[str(x[0])+'.jpg']+list(x[1]) for x in test_data]

train, val = train_test_split(formatted_train, train_size=0.9, random_state=15)

df_weeb_train = pd.DataFrame(train, columns = ['filename', '1girl', 'long_hair', 'blush', 'smile', 'short_hair'])
df_weeb_val = pd.DataFrame(val, columns = ['filename', '1girl', 'long_hair', 'blush', 'smile', 'short_hair'])
df_weeb_test = pd.DataFrame(formatted_test, columns = ['filename', '1girl', 'long_hair', 'blush', 'smile', 'short_hair'])

C:\Users\rapha\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [15]:
from keras.preprocessing.image import ImageDataGenerator
#Data augmentation
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   horizontal_flip=True,
                                   rotation_range=25,
                                   height_shift_range=0.1,
                                   width_shift_range=0.1)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# ['1girl', 'long_hair', 'blush', 'smile', 'short_hair']

train_generator = train_datagen.flow_from_dataframe(
        dataframe = df_weeb_train,
        directory = storage_dir,
        x_col = 'filename',
        y_col = taglist,
        target_size=(256, 256),
        batch_size=100,
        class_mode='other')

val_generator = val_datagen.flow_from_dataframe(
        dataframe = df_weeb_val,
        directory = storage_dir,
        x_col = 'filename',
        y_col = taglist,
        target_size=(256, 256),
        batch_size=50,
        class_mode='other')

test_generator = test_datagen.flow_from_dataframe(
        dataframe = df_weeb_test,
        directory = test_dir,
        x_col = 'filename',
        y_col = taglist,
        target_size=(256, 256),
        batch_size=50,
        class_mode='other')

Found 8853 images.
Found 984 images.
Found 982 images.


### Model proper: 5 tags

In [16]:
from keras import layers
from keras import models
from keras import optimizers
from keras import regularizers
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [17]:
from keras import layers
from keras import models
from keras import optimizers
from keras import regularizers
from keras import callbacks

callback = [EarlyStopping(monitor='val_loss', patience=2), ModelCheckpoint(filepath='weeb_current.h5', monitor='val_loss', save_best_only=True)]

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',input_shape=(256, 256, 3)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(rate=0.15))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(rate=0.15))

model.add(layers.Flatten())

model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(rate=0.4))

model.add(layers.Dense(5, activation='relu'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr = 0.001),
              metrics=['acc'])

In [18]:
# danbooru_tags = model.load_weights('weeb_ec2_lastrun.h5')

In [ ]:
danbooru_tags = model.fit_generator(
      train_generator,
      steps_per_epoch=150,
#       callbacks=callback,
      epochs=10,
      validation_data=val_generator,
      validation_steps=20)

In [37]:
model.save('weeb_ec2_lastrun.h5') #3 layers 32, 164, 128 conv, dropout 0.15 for each, dropout 0.4, 256 dense to 5 dense
with open('weeb_ec2.history', 'wb') as file:
    pickle.dump(danbooru_tags, file)

In [ ]:
#weeb_ec2: 15 epochs, 3 layers 32, 164, 128 conv, dropout 0.1 for each, dropout 0.5, 512 dense to 5 dense

## LDA model

In [22]:
with open('training_LDAv2.pkl', 'rb') as file:
    training_data_LDA = pickle.load(file)
with open('test_LDAv2.pkl', 'rb') as file:
    test_data_LDA = pickle.load(file)

In [23]:
formatted_train_LDA = [[str(x[0])+'.jpg']+list(x[2]) for x in training_data_LDA]
formatted_test_LDA = [[str(x[0])+'.jpg']+list(x[2]) for x in test_data_LDA]

train2, val2 = train_test_split(formatted_train_LDA, train_size=0.9, random_state=15)

df_weeb_train_LDA = pd.DataFrame(train2, columns = ['filename',0,1,2,3,4,5,6,7,8,9])
df_weeb_val_LDA = pd.DataFrame(val2, columns = ['filename',0,1,2,3,4,5,6,7,8,9])
df_weeb_test_LDA = pd.DataFrame(formatted_test_LDA, columns = ['filename',0,1,2,3,4,5,6,7,8,9])

C:\Users\rapha\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [24]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen_LDA = ImageDataGenerator(rescale=1./255, 
                                   horizontal_flip=True,
                                   rotation_range=25,
                                   height_shift_range=0.1,
                                   width_shift_range=0.1)

val_datagen_LDA = ImageDataGenerator(rescale=1./255)
test_datagen_LDA = ImageDataGenerator(rescale=1./255)
col_list = list(df_weeb_test.columns[1:])

# ['1girl', 'long_hair', 'blush', 'smile', 'short_hair']

train_generator_LDA = train_datagen_LDA.flow_from_dataframe(
        dataframe = df_weeb_train_LDA,
        directory = storage_dir,
        x_col = 'filename',
        y_col = [0,1,2,3,4,5,6,7,8,9],
        target_size=(256, 256),
        batch_size=20,
        class_mode='other')

val_generator_LDA = val_datagen_LDA.flow_from_dataframe(
        dataframe = df_weeb_val_LDA,
        directory = storage_dir,
        x_col = 'filename',
        y_col = [0,1,2,3,4,5,6,7,8,9],
        target_size=(256, 256),
        batch_size=20,
        class_mode='other')

test_generator_LDA = test_datagen_LDA.flow_from_dataframe(
        dataframe = df_weeb_test_LDA,
        directory = test_dir,
        x_col = 'filename',
        y_col = [0,1,2,3,4,5,6,7,8,9],
        target_size=(256, 256),
        batch_size=20,
        class_mode='other')

Found 8853 images.
Found 984 images.
Found 982 images.


In [26]:
from keras import layers
from keras import models
from keras import optimizers

model2 = models.Sequential()

model2.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(256, 256, 3)))

model2.add(layers.Conv2D(32, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))

model2.add(layers.Conv2D(64, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Dropout(rate=0.15))

model2.add(layers.Conv2D(128, (3, 3), activation='relu'))
model2.add(layers.MaxPooling2D((2, 2)))
model2.add(layers.Dropout(rate=0.15))

model2.add(layers.Flatten())

model2.add(layers.Dense(256, activation='relu'))
model2.add(layers.Dropout(rate=0.4))

model2.add(layers.Dense(10, activation='softmax'))

model2.compile(loss='mean_squared_error',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [ ]:
weeb_LDA = model2.fit_generator(
      train_generator_LDA,
      steps_per_epoch=150,
      epochs=10,
      validation_data=val_generator_LDA,
      validation_steps=20)

In [45]:
model2.save('weeb_LDA.h5')
with open('weeb_LDA.history', 'wb') as file:
    pickle.dump(weeb_LDA, file)